In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '34fd877d2ddb4305ab79222521608e23'
client_secret = '038aac820f624a37b72bbd83480da3bd'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
''' path = '/content/drive/MyDrive/Springboard/Capstone/BigML_Dataset_60ed7ff5e4279b24a20060d3.csv'
data_lyrics = pd.read_csv(path)
data_lyrics.dropna(inplace = True)
data_lyrics.reset_index(drop=True,inplace=True) '''

# Data wrangling 

In [ ]:
def find_track_id_database(database):

  """
  Function to find all the Spotify track ID's using the track name and artist of the lyrics dataset.
  This function is quite slow and the Spotify API doens't allow to run the whole dataset at once.
  Run this function multiple times over subsets of the data.
  """

  song_ids = []
  n = 1000
  len_data = len(database)

  for i in range(len(database)):

    results = sp.search(q= 'track: {} artist: {}'.format(database['Song'][i], database['Band'][i]), limit=1)

    if results['tracks']['items'] == []:
      song_ids.append(np.nan)
    else:
      track_id = results['tracks']['items'][0]['id']
      song_ids.append(track_id)

    if i % n == 0:
      print('{} of {} tracks'.format(i,len_data))
    
  database['id'] = song_ids
  database.dropna(inplace = True)
  database.reset_index(drop = True, inplace = True)

  database.to_csv('/content/drive/MyDrive/Springboard/Capstone/lyrics_id_notebooks/data_lyrics_ids_30000.csv')

  return database

In [ ]:
def data_lyrics_load():
  """
  Load the multiple .csv files created by running find_track_id_database on subsets of the lyrics dataset.
  Concatenate the files and output a concatenated dataframe with the track ID's and lyrics and a lookup table to link the ID's with the track names and artists.
  """

  paths = glob.glob('/content/drive/MyDrive/Springboard/Capstone/lyrics_id_notebooks/*.csv')

  dataframes = []
  for i in range(len(paths)):
    data = pd.read_csv(paths[i])
    dataframes.append(data)

  #concatenate seperate dataframes
  data_lyrics = pd.concat(dataframes)
  data_lyrics.drop_duplicates(subset='id', inplace=True)

  # Create lookup table to link id's and track title/artist
  id_lookup = data_lyrics[['Song','Band','id']]
  id_lookup.reset_index(drop=True, inplace=True)

  # Dataframe with song id's and lyrics
  data_lyrics = data_lyrics[['id','Lyrics']]
  data_lyrics.reset_index(drop=True, inplace=True)

  return data_lyrics, id_lookup

In [ ]:
def get_audio_features(dataset):

  """
  Function to get the Spotify audio features of a dataframe using the Spotify track ID's.
  """
  dataframes = []
  len_data = len(dataset)
  n = 10000

  for i in range(0, len(dataset), 100):
    list_ids = list(dataset['id'][i:i+100].values)
    audio_features = sp.audio_features(list_ids)
    audio_features = [i for i in audio_features if i]

    df_features = pd.DataFrame(audio_features)
    df_features = df_features[['id','danceability','energy', 'loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

    dataframes.append(df_features)

    if i % n == 0:
      print('{} of {} tracks'.format(i,len_data))

  data_lyrics_concat = pd.concat(dataframes)
  data_lyrics_features = dataset.merge(data_lyrics_concat, how='inner', on='id')

  data_lyrics_features.to_csv('/content/drive/MyDrive/Springboard/Capstone/data_lyrics_features.csv')

  return data_lyrics_features

# Data exploration